In [1]:
from datetime import datetime, timedelta
from pprint import pprint
from typing import Any, Dict, List, Optional, Union
from uuid import uuid4

import pystac
from pystac.utils import (
    StringEnum,
    datetime_to_str,
    get_required,
    map_opt,
    str_to_datetime,
)
from pystac.extensions.base import PropertiesExtension, ExtensionManagementMixin


This extension will extend a STAC Collection by adding these properties https://github.com/stac-extensions/ml-model#item-properties

Creating a class for the ML Model Extension

In [ ]:
class OrderEventType(StringEnum):
    SUBMITTED = "submitted"
    STARTED_PROCESSING = "started_processing"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"


class OrderEvent:
    properties: Dict[str, Any]

    def __init__(self, properties: Dict[str, Any]) -> None:
        self.properties = properties

    @property
    def event_type(self) -> OrderEventType:
        return get_required(self.properties.get("type"), self, "event_type")

    @event_type.setter
    def event_type(self, v: OrderEventType) -> None:
        self.properties["type"] = str(v)

    @property
    def timestamp(self) -> datetime:
        return str_to_datetime(
            get_required(self.properties.get("timestamp"), self, "timestamp")
        )

    @timestamp.setter
    def timestamp(self, v: datetime) -> None:
        self.properties["timestamp"] = datetime_to_str(v)

    def __repr__(self) -> str:
        return "<OrderEvent " f"type={self.event_type} " f"timestamp={self.timestamp}>"

    def apply(
        self,
        event_type: OrderEventType,
        timestamp: datetime,
    ) -> None:
        self.event_type = event_type
        self.timestamp = timestamp

    @classmethod
    def create(
        cls,
        event_type: OrderEventType,
        timestamp: datetime,
    ) -> "OrderEvent":
        oe = cls({})
        oe.apply(event_type=event_type, timestamp=timestamp)
        return oe

    def to_dict(self) -> Dict[str, Any]:
        return self.properties